In [1]:
#%reset -f
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import dask as da
import glob
import time
from datetime import datetime, timedelta as delta
from matplotlib import path
from matplotlib.collections import PolyCollection #for plots polygons as rasters
#for animation
import matplotlib.animation as animation
from copy import deepcopy
import matplotlib as mpl
import time
#import tqdm 
from tqdm import tqdm #to see progressbar for loops
from scipy.spatial import KDTree, cKDTree #c implementation is faster (to find neighbor)
%matplotlib inline
# to see the plot labels when using jupyter-lab dark mode
from matplotlib import style
#style.use('classic') #like matlab style (gray background, but white when saving figures)
#style.use('ggplot')
style.use('default') #the above styles problems with PolyCollection 
#print(style.available) #to see the styles 
import pickle #to save data as binary file .pkl
from windrose import WindroseAxes
import seaborn as sns #for heat maps of correlation and RMSD
from itertools import product #to make all possible permutations including repeated values
#
from scipy.interpolate import griddata #interpolation in space for non-uniform grids
from scipy.interpolate import interp1d #1d interp

#for projections
from pyproj import Proj, transform, Transformer

In [2]:
#inputs----------
#path of directories
home_dir = "/export/lv4/user/jfajardourbina/"
dir_vel = f"{home_dir}dws_ulf_getm_2D_depth_avg/data/velocity/" #vel data
dir_wind=f"{home_dir}dws_ulf_getm_2D_depth_avg/data/atmosphere/" #winds
dir_dws_bound=f"{home_dir}dws_ulf_getm_2D_depth_avg/experiments_post_proc/analysis_eulerian_data_36years/data_dws_boundaries/" #DWS boundarie with contour0
dir_topo=f"{home_dir}dws_ulf_getm_2D_depth_avg/experiments_post_proc/analysis_eulerian_data_36years/data_bathy_grid/" #topo data
savee='everyM2' #saving track data every m2
deploy='everyM2'#deploy set of particles every m2
minTsim=60 #mimimum time of simulation (days)
maxTsim=91 #maximum time of simulation (days)
dir_tracks = f"{homee}dws_ulf_getm_2D_depth_avg/experiments_post_proc/lagrangian_simulation_36years/exp-deployHighVolume_coords-xcyc_save-{savee}_deploy-{deploy}_Tsim-{minTsim}-{maxTsim}d/tracks/"
#files
file_dws_bound0="dws_boundaries_contour0.nc"
file_topo="DWS200m.2012.v03.nc"
file_vel="RE.DWS200m.uvz.20090301.nc" #any vel file
file_wind="UERRA.2009.nc4" #any wind file
#parameters
npa_per_dep=12967 #number of particles per deployment
m2=int(12.42*3600+2) #period in seconds
dx=400/1e3;dy=400/1e3 #particle grid reso
ref_time=np.datetime64("1980-01-01") #reference time for time interpolation
#
#paths for output data
dir_post_proc_data=f"{home_dir}dws_ulf_getm_2D_depth_avg/experiments_post_proc/lagrangian_simulation_36years/machine_learning_github/Lagrangian_ML/post_proc_data/"
dir_interp_wind="wind/"
file_interp_wind_root="wind_avg_during_1M2_and_interp_to_particle_grid_for_convlstm.nc"

In [3]:
#open topo file
dsto=xr.open_dataset(dir_topo+file_topo) 
xct0=dsto.xc.min().values/1e3; yct0=dsto.yc.min().values/1e3 #=(0,0)

#open DWS contours
dsb0=xr.open_dataset(dir_dws_bound+file_dws_bound0) 
bdr_dws0=dsb0.bdr_dws.values #points that define DWS with contour0

#open any vel data
dsv=xr.open_dataset(dir_vel+file_vel)
xcv=dsv.xc; ycv=dsv.yc; hv=dsv.bathymetry.load()
xcv0=xcv.min().values/1e3; ycv0=ycv.min().values/1e3
dsv.close()

#open any wind data
dsw=xr.open_dataset(dir_wind+file_wind) #winds
dsw.close()


#index from the model topo grid units from which the velocity y-axis start and finish
#the model of the simulation is shorter in the y-axis than the full topo domain
iy0=np.nonzero(dsto.yc.values==dsv.yc[0].values)[0][0]
print(iy0,dsto.yc.values[iy0],dsv.yc.values[0])
iy1=np.nonzero(dsto.yc.values==dsv.yc[-1].values)[0][0]
print(iy1,dsto.yc.values[iy1],dsv.yc.values[-1])

#create mask for islands
h=dsto.bathymetry; mask=h.where(np.isnan(h),1); mask=mask.where(np.isfinite(h),0)

#define the transformations---
#1)
#from epgs:28992(DWS) to epgs:4326(LatLon with WGS84 datum used by GPS and Google Earth)
proj = Transformer.from_crs('epsg:28992','epsg:4326',always_xy=True)
#2)
#from epgs:4326(LatLon with WGS84) to epgs:28992(DWS) 
inproj = Transformer.from_crs('epsg:4326','epsg:28992',always_xy=True)
#inproj_old=Proj("EPSG:28992") #old method (has errors 10-20m when contrast with the rotated coords)

#lon,lat to 28992(DWS)-projection--------------------

#bathymetry--------
xct=dsto.lonc.values;  yct=dsto.latc.values #lon,lat units
xctp,yctp,z = inproj.transform(xct,yct,xct*0.)
#[xctp,yctp] = inproj_old(xct,yct) #old method
xctp=(xctp)/1e3; yctp=(yctp)/1e3 
#first projected point to correct the coordinates of model local meter units
xctp0=xctp[0,0]; yctp0=yctp[0,0]

#wind original 2h-data--------
#use the full domain---
#usually problems when using local projection for big domains, but for our case no problems
xcw,ycw=np.meshgrid(dsw.lon.values,dsw.lat.values)
xcwp,ycwp,z = inproj.transform(xcw,ycw,xcw*0.)
xcwp=xcwp/1e3; ycwp=ycwp/1e3
#xcw=(xcw-xcw[0,0])/1e3; ycw=(ycw-ycw[0,0])/1e3
#use a short domain around DWS---
xcw2,ycw2=np.meshgrid(dsw.lon.sel(lon=slice(4,6.5)).values,dsw.lat.sel(lat=slice(52.5,53.75)).values)
xcwp2,ycwp2,z = inproj.transform(xcw2,ycw2,xcw2*0.)
xcwp2=(xcwp2)/1e3; ycwp2=(ycwp2)/1e3


#local meter model units to 28992(DWS)-projection and lon-lat--------------

#matrix rotation -17degrees-----
ang=-17*np.pi/180
angs=np.ones((2,2))
angs[0,0]=np.cos(ang); angs[0,1]=np.sin(ang)
angs[1,0]=-np.sin(ang); angs[1,1]=np.cos(ang)

#bathymetry----
#original topo points in meter
xct2,yct2=np.meshgrid(dsto.xc.values,dsto.yc.values)
xy=np.array([xct2.flatten(),yct2.flatten()]).T
#rotate
xyp=np.matmul(angs,xy.T).T/1e3
xyp0=xyp[0,:] #the first point in the bathy data in local meter units=0,0
#correct model units:
#1)substact the first model local point of the topo file, but give tha same as xyp0=[0,0]
#2)use the first projected point of the case (lon,lat model units to meter)
xyp=xyp-xyp0 
xyp[:,0]=xyp[:,0]+xctp0; xyp[:,1]=xyp[:,1]+yctp0 
xyp=np.reshape(xyp,(len(dsto.yc.values),len(dsto.xc.values),2))
xctp2=xyp[...,0]; yctp2=xyp[...,1] #km
#
#contrast projections (lon,lat model units to projection in meter) with rotated case
#around 0 meter diff with new method
#10 meter difference in average and maximum of 20 with old method
a=xctp-xctp2; b=yctp-yctp2
print(np.abs(a).max()*1e3, np.abs(b).max()*1e3, np.abs(a).mean()*1e3, np.abs(b).mean()*1e3) 

#contour0 of DWS------
#rotate
bdr_dws0p=np.matmul(angs,bdr_dws0.T).T/1e3
#correct model units:
#1)substact the first model local point of the topo file, but give tha same as xyp0=[0,0]
#2)use the first projected point of the case (lon,lat model units to meter)
bdr_dws0p=bdr_dws0p-xyp0 
bdr_dws0p[:,0]=bdr_dws0p[:,0]+xctp0; bdr_dws0p[:,1]=bdr_dws0p[:,1]+yctp0
#
#get coordinates in lon-lat units (WGS84 ) 
bdr_dws0_lon, bdr_dws0_lat, z = proj.transform(bdr_dws0p[:,0]*1e3,bdr_dws0p[:,1]*1e3, bdr_dws0p[:,1]*0.)


#28992(DWS)-projection to local meter model units--------------

#matrix rotation 17degrees-----
ang2=17*np.pi/180
angs2=np.ones((2,2))
angs2[0,0]=np.cos(ang2); angs2[0,1]=np.sin(ang2)
angs2[1,0]=-np.sin(ang2); angs2[1,1]=np.cos(ang2)

#batymetry---
xy=np.array([xctp.flatten(),yctp.flatten()]).T #km
#xy=xy-xy[0]
ny,nx=xctp.shape
#rotate
xyl=np.matmul(angs2,xy.T).T
xyl0=xyl[0,:] #the first point 
#correct model units:
#1)substact the first model local point of the topo file, but give tha same as xyp0=[0,0]
#2)use the first projected point of the case (lon,lat model units to meter)
xyl=xyl-xyl0 
xyl[:,0]=xyl[:,0]+xct0; xyl[:,1]=xyl[:,1]+yct0 
xyl=np.reshape(xyl,(ny,nx,2))
xctl=xyl[...,0]; yctl=xyl[...,1] #km

#errors in the position when using the local values get from transforamtion (around 0.1% of the grid size)
a=abs(xctl*1e3-dsto.xc.values)
b=abs(yctl.T*1e3-dsto.yc.values)
print('max, mean, max% errors in x (m)',a.max(), a.mean(), a.max()/200*100,"%") # max=12cm, mean=5cm
print('max, mean, max% errors in y (m)',b.max(), b.mean(), b.max()/200*100,"%") # max=32cm, mean=13cm

99 19800.0 19800.0
479 95800.0 95800.0
0.026940217722426496 0.3400088997977946 0.009365292291273685 0.1443332403121348
max, mean, max% errors in x (m) 0.12412859736627979 0.04687436167418282 0.062064298683139896 %
max, mean, max% errors in y (m) 0.32751821742022 0.13691407317063156 0.16375910871011 %


In [4]:
files_wind=sorted(glob.glob(f'{dir_wind}/**/*.nc4',recursive=True))

In [2]:
%%time
#all the years take 1.5h
for file_wind in tqdm(files_wind):

    year=int(str(file_wind)[-8:-4])
    print(year)

    #open wind data------

    dsw=xr.open_dataset(file_wind);dsw.close() #winds
    tw = dsw.time.values #contains data for the full 1st day of the next year

    #get times to avg wind-------

    #first track of this year---
    month_sim=1
    file_track=f'tracks_{year}{month_sim:02d}_coords-xcyc_save-{savee}_deploy-{deploy}_Tsim-{minTsim}-{maxTsim}d.nc'
    file_track_path=f'{dir_tracks}{year}/{file_track}'  
    dst=xr.open_dataset(file_track_path)
    t0=dst.time.isel(traj=0,obs=0).values 
    x0=dst.x.isel(traj=range(npa_per_dep),obs=0).values
    y0=dst.y.isel(traj=range(npa_per_dep),obs=0).values
    dst.close(); del dst

    #first track of the following year---
    if file_wind!=files_wind[-1]:
        file_track=f'tracks_{year+1}{month_sim:02d}_coords-xcyc_save-{savee}_deploy-{deploy}_Tsim-{minTsim}-{maxTsim}d.nc'
        file_track_path=f'{dir_tracks}{year+1}/{file_track}'  
        t1=xr.open_dataset(file_track_path).time.isel(traj=0,obs=0).values
    #last track of this year (for the final simulated month)---
    else: 
        #for the final year we can not open the next year simulation
        #we only have tracks until october, so we can get the wind for the last interval of displacement
        last_year_tracks=sorted(glob.glob(f'{dir_tracks}{year}/*.nc',recursive=True))
        end_month=len(last_year_tracks)
        file_track=f'tracks_{year}{end_month:02d}_coords-xcyc_save-{savee}_deploy-{deploy}_Tsim-{minTsim}-{maxTsim}d.nc'    
        file_track_path=f'{dir_tracks}{year}/{file_track}'  
        t1=xr.open_dataset(file_track_path).time.isel(traj=-1,obs=0).values + np.timedelta64(m2,'s')
    #
    #times to get wind data for this year---
    nt_int=283*2 #get wind data every 9.43 min (factor of m2=44714)
    #t_int: data until the next m2 after the last deployment
    #the next year for a particular year, or the same year for the final month (October) of the simulation 
    t_int=np.arange(t0,t1+np.timedelta64(1,'s'),nt_int,dtype='datetime64[s]') 
    t_dep=np.arange(t0,t1,m2,dtype='datetime64[s]') #only for this year
    

    #Get uw and vw avg during M2 interval of displacement and interpolated in particle grid points ------

    #interpolation in time-------

    #convert time to float
    t_int0=(t_int-ref_time) / np.timedelta64(1,'s')  #dates to interpolate (consistent with tracks)
    tw0=(tw-ref_time) / np.timedelta64(1,'s') #wind
    #interp
    f = interp1d(tw0, dsw.u10.values, axis=0, kind='linear'); u10_t_int=f(t_int0)
    f = interp1d(tw0, dsw.v10.values, axis=0, kind='linear'); v10_t_int=f(t_int0)
    
    #make avg during the interval of displacement (right border of interval open)---
    #
    nt_interval=int(m2/nt_int) #points in the m2 interval (right border of interval open)
    nt_mean=(len(t_int)-1)//nt_interval #final shape after mean in the interval
    u10_t_int_mean=np.reshape(u10_t_int[:-1,...],(nt_mean,nt_interval,len(dsw.lat),len(dsw.lon)))
    v10_t_int_mean=np.reshape(v10_t_int[:-1,...],(nt_mean,nt_interval,len(dsw.lat),len(dsw.lon))) 
    #vectorial mean---
    u10_t_int_mean=u10_t_int_mean.mean(axis=1)                       
    v10_t_int_mean=v10_t_int_mean.mean(axis=1)
    #
    #times are referenced with the date of deployment (the begin of the m2 interval of the displacement)
    t_mean=t_dep

    #Spatial interpolation of wind data using projected grid, and then to model xc-yc coordinates------

    #reshape---
    #using avg data in m2 interval
    ntw,nyw,nxw=u10_t_int_mean.shape
    u10=np.reshape(u10_t_int_mean,(ntw,nyw*nxw))
    v10=np.reshape(v10_t_int_mean,(ntw,nyw*nxw))

    #interpolation using model projected coordinates with cubic spline----
    #because our wind data is too low resolution in contrast with model grid
    #cubic spline interpolation seems more suitable
    points=np.array([xcwp.flatten(),ycwp.flatten()]).T
    u10 = np.moveaxis(griddata(points, u10.T, (xctp, yctp), method='cubic'),-1,0)
    v10 = np.moveaxis(griddata(points, v10.T, (xctp, yctp), method='cubic'),-1,0)
    np.sum(np.isnan(u10)),np.sum(np.isnan(v10))
    
    #projection of above data to model coordinates (+17deg rotation)----
    uv10=np.expand_dims(np.stack((u10,v10),axis=-1),-1) #(nt,ny,nx,2,1)
    uv10=np.squeeze(np.matmul(angs2,uv10)) #(2,2)x(nt,ny,nx,2,1) = (nt,ny,nx,2,1)
    u10,v10=uv10[...,0],uv10[...,1]; del uv10
    #explicity (same as above)
    #u10l= np.cos(ang2)*u10 + np.sin(ang2)*v10
    #v10l= -np.sin(ang2)*u10 + np.cos(ang2)*v10
    #u10=u10l; v10=v10l; del u10l,v10l
    
    #finally select data in domain of velocity----
    u10=u10[:,iy0:iy1+1,:]; v10=v10[:,iy0:iy1+1,:]

    # #don't save this file (we dont need it for ML)---
    # #save data in model xc,yc grid points (gird of the velocity field)---
    # dsout = xr.Dataset()
    # #global coords and attrs---
    # dsout.coords["time"] = t_mean #=t_dep
    # dsout["time"].attrs['description'] = 'initial date of the M2 average interval'
    # dsout.coords["yc"] = dsv.yc.values.astype("float32")
    # dsout["yc"].attrs['long_name'] = 'y'
    # dsout["yc"].attrs['units'] = 'm'
    # dsout.coords["xc"] = dsv.xc.values.astype("float32")
    # dsout["xc"].attrs['long_name'] = 'x'
    # dsout["xc"].attrs['units'] = 'm'
    # #
    # dsout.attrs["temporal_info"] = f"wind was averaged during the M2(={m2}s) interval of the net displacement, with right border of interval open"
    # dsout.attrs["spatial_info"] = "wind was cubic-interpolated into the local xc and yc model units"
    # #
    # #variables---
    # #
    # dsout["u10"] = (("time","yc","xc"),u10.astype("float32"))
    # dsout["u10"].attrs['long_name'] = 'interpolated wind in model x-direction'
    # dsout["u10"].attrs['units'] = 'm/s'
    # #
    # dsout["v10"] = (("time","yc","xc"),v10.astype("float32"))
    # dsout["v10"].attrs['long_name'] = 'interpolated wind in model y-direction'
    # dsout["v10"].attrs['units'] = 'm/s'
    # #
    # file_out_nc=f"{year}_avg_wind_during_1M2_and_interp_in_model_xcyc_grid.nc"
    # dir_out_nc="post_proc_data/wind/"
    # dsout.to_netcdf(dir_out_nc+file_out_nc)

    #Spatial interpolation of wind data into grid of displacements (with nearest neigh)------
    #Using drid data is identical to our nearest gridding method used for the net displacement

    #build grid (like the one of displacements)
    xmin=x0.min();xmax=x0.max();ymin=y0.min();ymax=y0.max()
    extend_grid=10 #so from particle min max positions extend grid 10*dx (to not have problems with convolution)
    xgrid=np.arange(xmin-dx*1e3*extend_grid,xmax+dx*1e3*(extend_grid+1),dx*1e3,dtype='float32')
    ygrid=np.arange(ymin-dy*1e3*extend_grid,ymax+dy*1e3*(extend_grid+1),dy*1e3,dtype='float32')
    xgrid0,ygrid0=np.meshgrid(xgrid,ygrid)

    #reshape---
    ntw,nyw,nxw=u10.shape
    u10=np.reshape(u10,(ntw,nyw*nxw))
    v10=np.reshape(v10,(ntw,nyw*nxw))

    #grid data with nearest----
    xx,yy=np.meshgrid(dsv.xc.values.astype("float32"),dsv.yc.values.astype("float32"))
    points=np.array([xx.flatten(),yy.flatten()]).T
    u10 = np.moveaxis(griddata(points, u10.T, (xgrid0, ygrid0), method='nearest'),-1,0)
    v10= np.moveaxis(griddata(points, v10.T, (xgrid0, ygrid0), method='nearest'),-1,0)
    np.sum(np.isnan(u10)),np.sum(np.isnan(v10))

    #save data---
    dsout1 = xr.Dataset()
    #global coords and attrs---
    dsout1.coords["time"] = t_mean #=t_dep
    dsout1["time"].attrs['description'] = 'initial date of the M2 average interval'
    dsout1.coords["y"] = ygrid.astype('float32')
    dsout1["y"].attrs['long_name'] = 'y'
    dsout1["y"].attrs['description'] = 'axis equal to the particle grid y-axis'
    dsout1["y"].attrs['units'] = 'm'
    dsout1.coords["x"] = xgrid.astype('float32')
    dsout1["x"].attrs['long_name'] = 'x'
    dsout1["x"].attrs['description'] = 'axis equal to the particle grid x-axis'
    dsout1["x"].attrs['units'] = 'm'
    #
    dsout1.attrs["temporal_info"] = f"Wind components were linearly interpolated to {nt_int}s (factor of M2={m2}s), and then averaged during the M2 interval of the net displacement."
    dsout1.attrs["spatial_info"] = "1) Wind components were interpolated with cubic-spline in EPSG:28992 projection. 2) Then projected to GETM model xc and yc local coordinates. 3) Finally, nearest-interpolated to xc-yc particle grid."    
    #
    #variables---
    #
    dsout1["u10"] = (("time","y","x"),u10.astype('float32'))
    dsout1["u10"].attrs['long_name'] = 'interpolated wind in x-direction'
    dsout1["u10"].attrs['units'] = 'm/s'
    #
    dsout1["v10"] = (("time","y","x"),v10.astype('float32'))
    dsout1["v10"].attrs['long_name'] = 'interpolated wind in y-direction'
    dsout1["v10"].attrs['units'] = 'm/s'
    #
    file_out_nc=f"{year}_{file_interp_wind_root}" 
    dir_out_nc=dir_post_proc_data+dir_interp_wind
    dsout1.to_netcdf(dir_out_nc+file_out_nc)
    dsout1.close(); del dsout1; del dsw

In [1]:
# #check above
# val=np.mean((u10g**2+v10g**2)**.5,axis=0)
# #
# #plot using model vel coordinates
# fig,ax= plt.subplots(1,1,figsize=(7,4))
# #aa=ax[1].contourf(xctl,yctl,values_int,levels=np.arange(0,8.5,.5),extend='max',cmap='jet') #wind correct orientation
# aa=ax.pcolormesh(xgrid/1e3,ygrid/1e3,val,vmin=0,vmax=8,shading='auto',cmap='jet') #wind correct orientation
# #ws2=ws_avg.sel(lon=slice(4,6.5),lat=slice(52.5,53.75)).values
# #aa=ax[1].contourf(xcw2,ycw2,ws2,levels=np.arange(0,8.5,.5),extend='max',cmap='jet') #wind correct orientation
# ax.contour(dsv.xc.values/1e3,dsv.yc.values/1e3,dsv.bathymetry,[7],colors='grey',linewidths=1) #bathy from topo file correct orientation
# ax.plot(bdr_dws0[:,0]/1e3,bdr_dws0[:,1]/1e3,'k') #DWS limits
# ax.axis('equal');ax.axis([30,155,30,80])
# #plt.axis([100,120,540,560])
# plt.colorbar(aa)
# ax.set_title('Cubic interpolation using original 1d xc,yc');